In [ ]:
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00


In [ ]:
import re
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate
import matplotlib.pyplot as plt

from torchtext.data.metrics import bleu_score

Importen un altre dataset extra per a que el vocabulari sigui més gran

In [ ]:
bleu = evaluate.load("bleu")

In [ ]:
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
dataset2 = datasets.load_dataset("bentrevett/multi30k")
train_data2, valid_data2, test_data2 = (
    dataset2["train"],
    dataset2["validation"],
    dataset2["test"],
)

In [ ]:
train_data2[0]['en']

'Two young, White males are outside near many bushes.'

In [ ]:
data2 = []
for frase in train_data2:
    frase_data = {'en':frase['en'], 'de':frase['de']}
    data2.append(frase_data)

In [ ]:
def eliminar_ccby(linea):
    indice_ccby = linea.find("CC-BY")
    if indice_ccby != -1:
        return linea[:indice_ccby]
    return linea

def separar_frases(linea):
    patron = r'(.+?[.!?])\s+([^A-Z]?[\s]+)?(.+)?'
    coincidencias = re.match(patron, linea)
    if coincidencias:
        frase1 = coincidencias.group(1)
        frase2 = coincidencias.group(3)
        return frase1.strip(), frase2.strip() if frase2 else None
    else:
        return None, None

def make_dataset(archivo_entrada):
    dataset = []
    with open(archivo_entrada, 'r') as f_in:
        for linea in f_in:
            linea_clean = eliminar_ccby(linea)
            a,b = separar_frases(linea_clean)
            if a!=None and b!=None:
              dataset.append({'en':a,'de':b})
    return dataset

In [ ]:
archivo_entrada = 'deu.txt'
data = make_dataset(archivo_entrada)
data += data2   # afegim el dataset extra

In [ ]:
len(data)

306227

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 63.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [ ]:
def tokenize_example(data, en_nlp, de_nlp, lower, sos_token, eos_token):

    delete = []
    for i in range(len(data)):

        try:
            en_tokens = [token.text for token in en_nlp.tokenizer(data[i]["en"])]
            de_tokens = [token.text for token in de_nlp.tokenizer(data[i]["de"])]
            if lower:
                en_tokens = [token.lower() for token in en_tokens]
                de_tokens = [token.lower() for token in de_tokens]
            en_tokens = [sos_token] + en_tokens + [eos_token]
            de_tokens = [sos_token] + de_tokens + [eos_token]

        except Exception as e:
            delete.append(i)

        else:
            data[i]['en_tokens'] = en_tokens   # només afegirem les dades si no salta cap error, sinó els elements afegits estaran bruts
            data[i]['de_tokens'] = de_tokens

    return data, delete


def process_in_batches(data, batch_size, en_nlp, de_nlp, lower, sos_token, eos_token):

    result = []
    for i in range(0, len(data), batch_size):  # 277 batches de 1000 elements

        batch = data[i:i+batch_size]
        processed_batch, delete = tokenize_example(batch, en_nlp, de_nlp, lower, sos_token, eos_token)
        for frase in delete:
            del(processed_batch[frase])
        result.extend(processed_batch)

    return result

In [ ]:
lower = True
sos_token = "<sos>"
eos_token = "<eos>"
batch_size = 256
dataset = data

inputs = [dataset, batch_size, en_nlp, de_nlp, lower, sos_token, eos_token]

Agafem un petit fragment del dataset per fer proves

In [ ]:
len(dataset)

306227

In [ ]:
process_in_batches(*inputs)

[{'en': 'Go.',
  'de': 'Geh.',
  'en_tokens': ['<sos>', 'go', '.', '<eos>'],
  'de_tokens': ['<sos>', 'geh', '.', '<eos>']},
 {'en': 'Hi.',
  'de': 'Hallo!',
  'en_tokens': ['<sos>', 'hi', '.', '<eos>'],
  'de_tokens': ['<sos>', 'hallo', '!', '<eos>']},
 {'en': 'Hi.',
  'de': 'Grüß Gott!',
  'en_tokens': ['<sos>', 'hi', '.', '<eos>'],
  'de_tokens': ['<sos>', 'grüß', 'gott', '!', '<eos>']},
 {'en': 'Run!',
  'de': 'Lauf!',
  'en_tokens': ['<sos>', 'run', '!', '<eos>'],
  'de_tokens': ['<sos>', 'lauf', '!', '<eos>']},
 {'en': 'Run.',
  'de': 'Lauf!',
  'en_tokens': ['<sos>', 'run', '.', '<eos>'],
  'de_tokens': ['<sos>', 'lauf', '!', '<eos>']},
 {'en': 'Wow!',
  'de': 'Potzdonner!',
  'en_tokens': ['<sos>', 'wow', '!', '<eos>'],
  'de_tokens': ['<sos>', 'potzdonner', '!', '<eos>']},
 {'en': 'Wow!',
  'de': 'Donnerwetter!',
  'en_tokens': ['<sos>', 'wow', '!', '<eos>'],
  'de_tokens': ['<sos>', 'donnerwetter', '!', '<eos>']},
 {'en': 'Duck!',
  'de': 'Kopf runter!',
  'en_tokens': ['<sos

sos = start of sentence,
eos = end of sentence,
unk = unknown,
pad = padding

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [ ]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]


all_en_tokens, all_de_tokens = [], []
for frase in dataset:
    all_en_tokens.append(frase['en_tokens'])
    all_de_tokens.append(frase['de_tokens'])


en_vocab = build_vocab_from_iterator(
    all_en_tokens,
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = build_vocab_from_iterator(
    all_de_tokens,
    min_freq=min_freq,
    specials=special_tokens,
)

Com a primera conclusió podem dir que l'alemany es un idioma més ric en paraules, ja que es fan servir gairebé el doble en un dataset de 277k frases

In [ ]:
words_en, words_de = en_vocab.get_itos(), de_vocab.get_itos()
len(words_en), len(words_de)

(14645, 27559)

In [ ]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

In [ ]:
def numericalize_example(data, en_vocab, de_vocab):
    for i in range(len(data)):
        en_ids = en_vocab.lookup_indices(data[i]["en_tokens"])
        de_ids = de_vocab.lookup_indices(data[i]["de_tokens"])
        data[i]['en_ids'] = en_ids
        data[i]['de_ids'] = de_ids
    return data

In [ ]:
dataset = numericalize_example(dataset, en_vocab, de_vocab)

L'equivalència token - índex és correcte

In [ ]:
for i in range(len(dataset)):
    en_ids = dataset[i]['en_ids']
    de_ids = dataset[i]['de_ids']
    dataset[i]['en_ids'] = torch.tensor(en_ids)
    dataset[i]['de_ids'] = torch.tensor(de_ids)

In [ ]:
dataset = dataset[:(batch_size*32)]

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)  #Fem servir padding com relleno
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

Posem un número de batch y un test_size que sigui divisible per batch_size per a que quadri bé

In [ ]:
batch_size = 256
test_size = batch_size*8

random.shuffle(dataset)
test_set = dataset[:test_size]
train_set = dataset[:test_size]

train_data_loader = get_data_loader(train_set, batch_size, pad_index, shuffle=True)
test_data_loader = get_data_loader(test_set, batch_size, pad_index, shuffle=True)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim, bidirectional=True)
        self.fc = nn.Linear(encoder_hidden_dim * 2, decoder_hidden_dim)
        self.dropout = nn.Dropout(dropout)  # recordem: dropout desactiva un % de neurones i així evita l'overfitting

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, hidden = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        # hidden [-2, :, : ] is the last of the forwards RNN
        # hidden [-1, :, : ] is the last of the backwards RNN
        # initial decoder hidden is final hidden state of the forwards and backwards
        # encoder RNNs fed through a linear layer
        hidden = torch.tanh(
            self.fc(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        )
        # outputs = [src length, batch size, encoder hidden dim * 2]
        # hidden = [batch size, decoder hidden dim]
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
        self.attn_fc = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim, decoder_hidden_dim
        )
        self.v_fc = nn.Linear(decoder_hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden = [batch size, decoder hidden dim]
        # encoder_outputs = [src length, batch size, encoder hidden dim * 2]
        batch_size = encoder_outputs.shape[1]
        src_length = encoder_outputs.shape[0]
        # repeat decoder hidden state src_length times
        hidden = hidden.unsqueeze(1).repeat(1, src_length, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # hidden = [batch size, src length, decoder hidden dim]
        # encoder_outputs = [batch size, src length, encoder hidden dim * 2]
        energy = torch.tanh(self.attn_fc(torch.cat((hidden, encoder_outputs), dim=2)))
        # energy = [batch size, src length, decoder hidden dim]
        attention = self.v_fc(energy).squeeze(2)
        # attention = [batch size, src length]
        return torch.softmax(attention, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(
        self,
        output_dim,
        embedding_dim,
        encoder_hidden_dim,
        decoder_hidden_dim,
        dropout,
        attention,
    ):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.GRU((encoder_hidden_dim * 2) + embedding_dim, decoder_hidden_dim)
        self.fc_out = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim + embedding_dim, output_dim
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        # input = [batch size]
        # hidden = [batch size, decoder hidden dim]
        # encoder_outputs = [src length, batch size, encoder hidden dim * 2]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        a = self.attention(hidden, encoder_outputs)
        # a = [batch size, src length]
        a = a.unsqueeze(1)
        # a = [batch size, 1, src length]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # encoder_outputs = [batch size, src length, encoder hidden dim * 2]
        weighted = torch.bmm(a, encoder_outputs)
        # weighted = [batch size, 1, encoder hidden dim * 2]
        weighted = weighted.permute(1, 0, 2)
        # weighted = [1, batch size, encoder hidden dim * 2]
        rnn_input = torch.cat((embedded, weighted), dim=2)
        # rnn_input = [1, batch size, (encoder hidden dim * 2) + embedding dim]
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        # output = [seq length, batch size, decoder hid dim * n directions]
        # hidden = [n layers * n directions, batch size, decoder hid dim]
        # seq len, n layers and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, decoder hidden dim]
        # hidden = [1, batch size, decoder hidden dim]
        # this also means that output == hidden
        assert (output == hidden).all()
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        # prediction = [batch size, output dim]
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        batch_size = src.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # encoder_outputs is all hidden states of the input sequence, back and forwards
        # hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
        # outputs = [src length, batch size, encoder hidden dim * 2]
        # hidden = [batch size, decoder hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden state and all encoder hidden states
            # receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, decoder hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

Valors amb el que anirem probant per millorar l'eficiència del model

In [ ]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
encoder_hidden_dim = 512
decoder_hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

attention = Attention(encoder_hidden_dim, decoder_hidden_dim)

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    decoder_dropout,
    attention,
)

model = Seq2Seq(encoder, decoder, device).to(device)

Cada paràmetre representa el weight o bias d'una capa (ih=input, hh=hidden) en el recorregut d'anada per calcular el loss o el de tornada que actualitza el gradient (reverse)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if "weight" in name:
            nn.init.normal_(param.data, mean=0, std=0.01)  # pesos molt propers a 0, desviació del 1%, perill de vanishing
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(23055, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_fc): Linear(in_features=1536, out_features=512, bias=True)
      (v_fc): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(12327, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=12327, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad),model.parameters()

a,b = count_parameters(model)
suma=0
for x in b:
  print(x)

print(f"The model has {a:,} trainable parameters",suma)

Parameter containing:
tensor([[ 0.0016, -0.0081, -0.0088,  ..., -0.0018, -0.0180,  0.0046],
        [-0.0224,  0.0043, -0.0046,  ...,  0.0101,  0.0085, -0.0100],
        [ 0.0169,  0.0099, -0.0025,  ..., -0.0053,  0.0020,  0.0102],
        ...,
        [-0.0053,  0.0179, -0.0115,  ..., -0.0124,  0.0020,  0.0083],
        [-0.0042, -0.0027, -0.0184,  ...,  0.0135,  0.0009,  0.0053],
        [ 0.0104, -0.0204,  0.0052,  ...,  0.0015,  0.0192, -0.0018]],
       requires_grad=True)
Parameter containing:
tensor([[-4.5452e-03,  7.1882e-04, -4.2238e-03,  ..., -2.0029e-02,
         -4.9447e-03, -1.9642e-04],
        [-8.7466e-03,  5.6466e-03,  1.0656e-02,  ..., -9.4040e-03,
          1.7749e-02, -4.0673e-02],
        [-6.4818e-03, -2.0398e-02, -8.2612e-05,  ...,  1.6728e-02,
         -2.2324e-02, -1.1203e-02],
        ...,
        [-1.4955e-02, -9.8210e-03, -6.6262e-03,  ..., -7.9117e-03,
         -3.9034e-05, -1.6107e-02],
        [-1.6784e-03,  6.6237e-03, -1.0110e-02,  ...,  5.8373e-03,
   

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

Totes les primeres files de cada batch estan plenes de 2, com l'inici de tots els en_ids o de_ids, que representa el eos

In [ ]:
def train_seq2seq(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["de_ids"].to(device)   # device és gpu collab ja que és on fem les probes per no gasta hores de mv azure
        trg = batch["en_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)


def evaluate_seq2seq(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    all_predictions, all_references = [], []

    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()

    avg_loss = epoch_loss / len(data_loader)
    return avg_loss

Ara hem de dividir el conjunt de dades que hem processat en train, test per entrenar i validar correctament la xarxa neuronal

In [ ]:
n_epochs = 2
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):  #tqdm per veure el processament del bucle, ja que tardará molt
    train_loss = train_seq2seq(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_seq2seq(
        model,
        test_data_loader,
        criterion,
        device,
    )

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut3-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

 50%|█████     | 1/2 [01:38<01:38, 98.40s/it]

	Train Loss:   9.619 | Train PPL: 15041.582
	Valid Loss:   9.541 | Valid PPL: 13922.135


100%|██████████| 2/2 [03:21<00:00, 100.78s/it]

	Train Loss:   9.634 | Train PPL: 15271.364
	Valid Loss:   9.544 | Valid PPL: 13964.689


Hem de executar això per declarar els inputs que tindrà el model abans de fer el load

In [ ]:
input_dim = len(de_vocab)  #21491
output_dim = len(en_vocab)  #11919
encoder_embedding_dim = 256
decoder_embedding_dim = 256
encoder_hidden_dim = 512
decoder_hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

attention = Attention(encoder_hidden_dim, decoder_hidden_dim)

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    decoder_dropout,
    attention,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
print(input_dim,output_dim)

In [ ]:
model.load_state_dict(torch.load('tut3-model.pt'))
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(27559, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_fc): Linear(in_features=1536, out_features=512, bias=True)
      (v_fc): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(14645, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=14645, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
test_loss = evaluate_seq2seq(model, test_data_loader, criterion, device)
print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

In [ ]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            de_tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            de_tokens = [token for token in sentence]
        if lower:
            de_tokens = [token.lower() for token in de_tokens]
        de_tokens = [sos_token] + de_tokens + [eos_token]
        ids = de_vocab.lookup_indices(de_tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        encoder_outputs, hidden = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        attentions = torch.zeros(max_output_length, 1, len(ids))
        for i in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, attention = model.decoder(
                inputs_tensor, hidden, encoder_outputs
            )
            attentions[i] = attention
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        en_tokens = en_vocab.lookup_tokens(inputs)
    return en_tokens, de_tokens, attentions[: len(en_tokens) - 1]

In [ ]:
def plot_attention(sentence, translation, attention):
    fig, ax = plt.subplots(figsize=(10, 10))
    attention = attention.squeeze(1).numpy()
    cax = ax.matshow(attention, cmap="bone")
    ax.set_xticks(ticks=np.arange(len(sentence)), labels=sentence, rotation=90, size=15)
    translation = translation[1:]
    ax.set_yticks(ticks=np.arange(len(translation)), labels=translation, size=15)
    plt.show()
    plt.close()

In [ ]:
i = 1
sentence = data[i]["de"]
expected_translation = data[i]["en"]
sentence, expected_translation

('Hallo!', 'Hi.')

In [ ]:
translation, sentence_tokens, attention = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

print(translation)
print(expected_translation)

['<sos>', 'funny', 'faced', 'requests', 'campsite', 'apologizing', 'racked', 'centered', 'procedure', 'lovely', 'curriculum', 'disagreement', 'bars', 'flirt', 'profitable', 'possibly', 'frame', 'assigned', 'referred', 'journalist', 'flipping', 'sheets', 'fill', 'hayes', 'or', 'handball']
Hi.


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00


In [ ]:
import wandb

# Inicializar wandb
api_key = "1d1e143a3eefd6e21e71cefd94bfa81337fc22e1" # your api key
wandb.login(key = api_key)
wandb.init(project="uab-grup14")


# Definir el optimizador y la función de pérdida
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

# Función de entrenamiento
def train_seq2seq(model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["de_ids"].to(device)
        trg = batch["en_ids"].to(device)
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

# Función de evaluación
def evaluate_seq2seq(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            output = model(src, trg, 0)  # turn off teacher forcing
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

# Función tokenizadora para translation (get bleu metric)
def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: brunoleon8 (uab-grup14). Use `wandb login --relogin` to force relogin


In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [160]:
n_epochs = 3
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")


for epoch in tqdm.tqdm(range(n_epochs)):

    train_loss = train_seq2seq(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )

    valid_loss = evaluate_seq2seq(
        model,
        test_data_loader,
        criterion,
        device,
    )

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "best_model.pt")


    translations = [
    translate_sentence(
        frase["de"],
        model,
        en_nlp,
        de_nlp,
        en_vocab,
        de_vocab,
        lower,
        sos_token,
        eos_token,
        device,
    )[0] for frase in tqdm.tqdm(test_set)]

    predictions = [" ".join(translation[1:-1]) for translation in translations]
    references = [[example["en"]] for example in test_set]

    tokenizer_fn = get_tokenizer_fn(en_nlp, lower)
    results = bleu.compute(predictions=predictions, references=references, tokenizer=tokenizer_fn)

    # Log metrics to wandb
    for key, value in results.items():
      wandb.log({key: value})
    wandb.log({
        'Train Loss': train_loss,
        'Valid Loss': valid_loss,
        'Train PPL': np.exp(train_loss),
        'Valid PPL': np.exp(valid_loss)
    })

    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")
    print(f"\tBleu: {results}")

# Finalizar wandb
wandb.finish()


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2048 [00:00<?, ?it/s]

  0%|          | 2/2048 [00:00<02:20, 14.53it/s]

  0%|          | 4/2048 [00:00<02:27, 13.89it/s]

  0%|          | 6/2048 [00:00<02:18, 14.77it/s]

  0%|          | 8/2048 [00:00<02:11, 15.49it/s]

  0%|          | 10/2048 [00:00<02:08, 15.88it/s]

  1%|          | 12/2048 [00:00<02:04, 16.30it/s]

  1%|          | 14/2048 [00:00<02:02, 16.64it/s]

  1%|          | 16/2048 [00:00<01:59, 16.95it/s]

  1%|          | 18/2048 [00:01<02:00, 16.83it/s]

  1%|          | 20/2048 [00:01<01:59, 16.94it/s]

  1%|          | 22/2048 [00:01<01:57, 17.17it/s]

  1%|          | 24/2048 [00:01<01:59, 16.90it/s]

  1%|▏         | 26/2048 [00:01<01:58, 17.12it/s]

  1%|▏         | 28/2048 [00:01<01:57, 17.13it/s]

  1%|▏         | 30/2048 [00:01<01:58, 17.08it/s]

  2%|▏         | 32/2048 [00:01<01:58, 16.95it/s]

  2%|▏         | 34/2048 [00:02<01:59, 16.87it/s]

  2%|▏         | 36/2048 [00:02<01:59, 16.81it/s]

  2%

	Train Loss:   6.070 | Train PPL: 432.750
	Valid Loss:   4.172 | Valid PPL:  64.857
	Bleu: {'bleu': 0.0, 'precisions': [0.2821044921875, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0123578843302026, 'translation_length': 8192, 'reference_length': 8092}




  0%|          | 0/2048 [00:00<?, ?it/s]

  0%|          | 2/2048 [00:00<01:50, 18.52it/s]

  0%|          | 4/2048 [00:00<01:54, 17.79it/s]

  0%|          | 6/2048 [00:00<01:59, 17.16it/s]

  0%|          | 8/2048 [00:00<02:00, 16.92it/s]

  0%|          | 10/2048 [00:00<02:00, 16.84it/s]

  1%|          | 12/2048 [00:00<02:02, 16.63it/s]

  1%|          | 14/2048 [00:00<02:00, 16.89it/s]

  1%|          | 16/2048 [00:00<01:59, 17.05it/s]

  1%|          | 18/2048 [00:01<01:58, 17.13it/s]

  1%|          | 20/2048 [00:01<01:57, 17.33it/s]

  1%|          | 22/2048 [00:01<01:55, 17.50it/s]

  1%|          | 24/2048 [00:01<01:56, 17.41it/s]

  1%|▏         | 26/2048 [00:01<01:56, 17.36it/s]

  1%|▏         | 28/2048 [00:01<01:55, 17.42it/s]

  1%|▏         | 30/2048 [00:01<01:58, 17.08it/s]

  2%|▏         | 32/2048 [00:01<01:57, 17.20it/s]

  2%|▏         | 34/2048 [00:01<01:56, 17.24it/s]

  2%|▏         | 36/2048 [00:02<01:55, 17.37it/s]

  2%|▏         | 38/2048 [00:02<01:55, 17.

	Train Loss:   3.880 | Train PPL:  48.424
	Valid Loss:   3.606 | Valid PPL:  36.834
	Bleu: {'bleu': 0.0, 'precisions': [0.3124456048738033, 0.008340283569641367, 0.0, 0.0], 'brevity_penalty': 0.9939262660172782, 'length_ratio': 0.9939446366782007, 'translation_length': 8043, 'reference_length': 8092}


 67%|██████▋   | 2/3 [08:11<04:05, 245.88s/it]


KeyboardInterrupt: 